# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [1]:
from applets.helpers import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def add_mod_FP(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def add_mod_N(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A60000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [2]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [3]:
app = Calculator(connection)
app.select()

b''

In [4]:
d = b'hello'
k = b'key'

In [5]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [6]:
import hashlib, hmac
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [7]:
FP = 2**256 - 2**32 - 977
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [8]:
def to_bytes(n:int):
    return n.to_bytes(32, 'big')

In [9]:
# random check
import random
for i in range(10):
    a = random.randint(1,N)
    b = random.randint(1,N)
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4
5
6
7
8
9


In [10]:
# edge cases - slightly less than mod, slightly more, equal and overflow
a = FP-20
for i, b in enumerate([10,23,20,10500, 1020203124]):
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4


In [11]:
from embit import bip32, ec
import os
seed = os.urandom(64)
root = bip32.HDKey.from_seed(seed)
# without prefix, depth, root fingerprint and index
root.serialize()[13:].hex()

'8525a11ab29da6dcd2daa80e80fd9087be0676b0dbd950494d52cf9a58ecd1b80079f431b6d40f9047887479d2c6e8620d330de77eec241687532458e5f3cfad27'

In [12]:
res = root.child(10, hardened=False).serialize()[13:].hex()
res

'82f48a660a2cfeb3ab83bd8d01a64b7422a80c31b8ef654468a939289b529c42001c98b8e68b59661bd22d32bc1de0b097597659848d9b2e1cff41036608168022'

In [13]:
data = bytes([65])+root.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0A90000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'82f48a660a2cfeb3ab83bd8d01a64b7422a80c31b8ef654468a939289b529c42001c98b8e68b59661bd22d32bc1de0b097597659848d9b2e1cff41036608168022'

In [14]:
# point addition
p = root.to_public().key
p.compressed = False
print(p.sec().hex())
pp = ec.PublicKey.parse(p.serialize())
ec.secp256k1.ec_pubkey_tweak_add(pp._point, b'4'*32)
res = pp.sec().hex()
print(res)

04862334be4e8f0794262d9ab64e51ad330472dcdb269e8827a776e8207baaa1beb84f438abb6d98089fc450020366b5472aae4a33282f72625c56259e8e86c628
0437b83b5e6caf0676b44aed73d2fba76a73660801c4a4fe0f30306c735dbd12649876c267ac1060df9dabd66186af8f4611526484aa902271e55fad8b124229d7


In [15]:
data = bytes([32])+b'4'*32+bytes([65])+p.sec()
r = "B0A70000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'0437b83b5e6caf0676b44aed73d2fba76a73660801c4a4fe0f30306c735dbd12649876c267ac1060df9dabd66186af8f4611526484aa902271e55fad8b124229d7'

In [16]:
p3 = ec.PublicKey.parse(ec.secp256k1.ec_pubkey_serialize(ec.secp256k1.ec_pubkey_combine(p._point,pp._point)))
p3.compressed = False
res = p3.sec().hex()
res

'04721ab66ba7f127719507694fd52696f91898f415d710fdd8f7b8d548e1d764d2013d6f0440482bfffedfef1a7dc5353408b27089f046de208c86f4c70f47613b'

In [17]:
data = bytes([65])+pp.sec()+bytes([65])+p.sec()
r = "B0A80000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'04721ab66ba7f127719507694fd52696f91898f415d710fdd8f7b8d548e1d764d2013d6f0440482bfffedfef1a7dc5353408b27089f046de208c86f4c70f47613b'

In [18]:
xpub = root.to_public()
xpub.serialize()[13:].hex()

'8525a11ab29da6dcd2daa80e80fd9087be0676b0dbd950494d52cf9a58ecd1b802862334be4e8f0794262d9ab64e51ad330472dcdb269e8827a776e8207baaa1be'

In [19]:
res=xpub.child(10, hardened=False).serialize()[13:].hex()
res

'82f48a660a2cfeb3ab83bd8d01a64b7422a80c31b8ef654468a939289b529c4202b20bff72b0ff4ad5185fc88eb0d203b267879466392be2385ca96752ceba551d'

In [20]:
fullpub = ec.PublicKey.parse(xpub.key.serialize())
fullpub.compressed=False
data = bytes([65])+xpub.serialize()[13:]+bytes([4,0x00,0,0,10])+bytes([65])+fullpub.sec()
r = "B0AA0000"+(bytes([len(data)])+data).hex().upper()
res2=app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh",res2)
res2

'82f48a660a2cfeb3ab83bd8d01a64b7422a80c31b8ef654468a939289b529c4202b20bff72b0ff4ad5185fc88eb0d203b267879466392be2385ca96752ceba551d'

In [21]:
import hashlib
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 2048, 64).hex())
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 100, 64).hex())

82218bfe339fa8a349423df85eaf89728b4e7dc02360e8c6feec346a8e46232db544f6f2d2fff1430e53f76c0adafd9a2087652b7b943fdc07c0cff4c74252e2
038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f


In [22]:
import time
time.time()

1586574010.268832

In [23]:
t0 = time.time()
# 2048 iterations -> 0x8000
iterations = 100
data = bytes([2])+iterations.to_bytes(2,'big')+bytes([3])+b'pwd'+bytes([4])+b'salt'
r = "B0A00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())
dt = time.time()-t0
print(f"{dt} seconds for {iterations} iterations.\nFor 2048 iterations it will be {dt*2048/iterations} seconds")

038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f
3.2884280681610107 seconds for 100 iterations.
For 2048 iterations it will be 67.3470068359375 seconds


In [24]:
connection.disconnect()